In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set visual style
sns.set(style="whitegrid")

# 1. Load Data
try:
    df = pd.read_csv('../data/raw/Fraud_Data.csv')
    print("✅ Fraud_Data loaded successfully.")
except FileNotFoundError:
    print("❌ File not found. Check path.")

# 2. Data Cleaning & Quality Check [cite: 113-116]
print(f"Initial Shape: {df.shape}")

# A. Convert Timestamps to Datetime objects
df['signup_time'] = pd.to_datetime(df['signup_time'])
df['purchase_time'] = pd.to_datetime(df['purchase_time'])

# B. Check for Duplicates
dupes = df.duplicated().sum()
if dupes > 0:
    print(f"⚠️ Found {dupes} duplicates. Removing them...")
    df.drop_duplicates(inplace=True)
else:
    print("✅ No duplicates found.")

# C. Check for Missing Values
missing = df.isnull().sum()
print(f"\nMissing Values:\n{missing[missing > 0]}")
# Note: If no missing values print, we are good.
# If 'device_id' or others have missing, we must decide to drop or impute.

# 3. Exploratory Data Analysis (EDA)

# A. Class Distribution (The Imbalance) [cite: 120]
fraud_count = df['class'].value_counts()
fraud_pct = df['class'].value_counts(normalize=True) * 100

print("\n--- Class Distribution ---")
print(fraud_count)
print(f"\nFraud Rate: {fraud_pct[1]:.2f}%")

print("\nColumn names:")
print(df.columns.tolist())

print("\nData types:")
print(df.dtypes)

print("\nMissing values per column:")
print(df.isnull().sum())

print("\nDuplicate rows:")
print(df.duplicated().sum())



In [ ]:
# Convert timestamps
df["signup_time"] = pd.to_datetime(df["signup_time"], errors="coerce")
df["purchase_time"] = pd.to_datetime(df["purchase_time"], errors="coerce")

print("\nNull timestamps after parsing:")
print(df[["signup_time", "purchase_time"]].isnull().sum())
# Time sanity: purchase should not happen before signup
invalid_time_order = df[df["purchase_time"] < df["signup_time"]]

print(f"\nTransactions where purchase_time < signup_time: {len(invalid_time_order)}")

if len(invalid_time_order) > 0:
    display(invalid_time_order.head())

In [ ]:
df["time_since_signup_seconds"] = (
    df["purchase_time"] - df["signup_time"]
).dt.total_seconds()

print("\nTime since signup summary (seconds):")
print(df["time_since_signup_seconds"].describe())

print("\nNegative or zero time_since_signup values:")
print((df["time_since_signup_seconds"] <= 0).sum())


In [ ]:

plt.figure(figsize=(6, 4))
sns.countplot(x='class', data=df)
plt.title('Class Distribution: Fraud (1) vs Non-Fraud (0)')
plt.show()

# B. Univariate Analysis: Purchase Value [cite: 118]
plt.figure(figsize=(10, 5))
sns.histplot(df['purchase_value'], bins=50, kde=True)
plt.title('Distribution of Purchase Value')
plt.show()

# C. Bivariate Analysis: Purchase Value vs Class [cite: 119]
# Do fraudsters spend more?
plt.figure(figsize=(10, 5))
sns.boxplot(x='class', y='purchase_value', data=df)
plt.title('Purchase Value by Class')
plt.show()

# D. Temporal Analysis (Quick check on time)
# Is there a gap between signup and purchase?
df['time_diff'] = (df['purchase_time'] - df['signup_time']).dt.total_seconds() / 3600 # in hours

plt.figure(figsize=(10, 5))
sns.histplot(data=df, x='time_diff', hue='class', kde=True, bins=50, common_norm=False)
plt.title('Time Difference (Signup vs Purchase) by Class')
plt.xlim(0, 48) # Zooming in on the first 48 hours to see immediate attacks
plt.show()

In [ ]:
# --- 4. Categorical Analysis ---
# We want to see if the Fraud Rate (Class 1) is higher for certain browsers or sources.

fig, axes = plt.subplots(1, 2, figsize=(18, 6))

# Browser Analysis
sns.countplot(x='browser', hue='class', data=df, ax=axes[0])
axes[0].set_title('Fraud by Browser')
axes[0].tick_params(axis='x', rotation=45)

# Source Analysis
sns.countplot(x='source', hue='class', data=df, ax=axes[1])
axes[1].set_title('Fraud by Source')

plt.tight_layout()
plt.show()

# --- 5. "Shared Entity" Analysis (Critical for Fraud) ---
# Fraudsters often use the same device or IP to make multiple fake accounts.

# A. Devices used by multiple users
device_counts = df.groupby('device_id')['user_id'].count()
print("\n--- Device ID Analysis ---")
print(f"Total unique devices: {len(device_counts)}")
print(f"Devices used by > 1 user: {len(device_counts[device_counts > 1])}")

# Let's plot the fraud rate against the number of users per device
# We merge the counts back to the main df
df['device_user_count'] = df['device_id'].map(device_counts)

plt.figure(figsize=(10, 5))
sns.barplot(x='device_user_count', y='class', data=df, errorbar=None)
plt.title('Fraud Rate vs. Number of Users on Same Device')
plt.xlabel('Number of Users Sharing Device')
plt.ylabel('Fraud Probability')
plt.show()

# B. IPs used by multiple users
ip_counts = df.groupby('ip_address')['user_id'].count()
df['ip_user_count'] = df['ip_address'].map(ip_counts)

print("\n--- IP Address Analysis ---")
print(f"IPs used by > 1 user: {len(ip_counts[ip_counts > 1])}")

plt.figure(figsize=(10, 5))
sns.barplot(x='ip_user_count', y='class', data=df, errorbar=None)
plt.title('Fraud Rate vs. Number of Users on Same IP')
plt.xlabel('Number of Users Sharing IP')
plt.ylabel('Fraud Probability')
plt.show()

# --- 6. Age Analysis ---
plt.figure(figsize=(10, 5))
sns.kdeplot(data=df[df['class']==0]['age'], label='Not Fraud', fill=True)
sns.kdeplot(data=df[df['class']==1]['age'], label='Fraud', fill=True)
plt.title('Age Distribution by Class')
plt.legend()
plt.show()

In [ ]:
# Ensure timestamps are datetime
df["signup_time"] = pd.to_datetime(df["signup_time"])
df["purchase_time"] = pd.to_datetime(df["purchase_time"])

# Time-based features
df["hour_of_day"] = df["purchase_time"].dt.hour
df["day_of_week"] = df["purchase_time"].dt.dayofweek  # 0=Monday

df[["hour_of_day", "day_of_week"]].describe()

#visualize fraud patterns
plt.figure(figsize=(8, 5))
sns.countplot(x="hour_of_day", hue="class", data=df)
plt.title("Fraud Distribution by Hour of Day")
plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
sns.kdeplot(
    df[df["class"] == 0]["time_since_signup_seconds"],
    label="Legitimate",
    fill=True
)
sns.kdeplot(
    df[df["class"] == 1]["time_since_signup_seconds"],
    label="Fraud",
    fill=True
)
plt.title("Time Since Signup Distribution by Class")
plt.xlabel("Seconds Since Signup")
plt.legend()
plt.show()


In [ ]:
# --- 6. Geolocation Analysis ---

# 1. Load the IP dataset
try:
    ip_df = pd.read_csv('../data/raw/IpAddress_to_Country.csv')
    print("✅ IpAddress_to_Country loaded successfully.")
except FileNotFoundError:
    print("❌ File not found.")

# 2. Convert IP addresses to integers for accurate comparison [cite: 380]
# Note: IP addresses in Fraud_Data are floats, we convert to int.
df['ip_address'] = df['ip_address'].astype(int)
ip_df['lower_bound_ip_address'] = ip_df['lower_bound_ip_address'].astype(int)
ip_df['upper_bound_ip_address'] = ip_df['upper_bound_ip_address'].astype(int)

# 3. Efficient Merge using merge_asof
# We must sort both dataframes by the key we are merging on (IP)
df = df.sort_values('ip_address')
ip_df = ip_df.sort_values('lower_bound_ip_address')

# merge_asof matches the 'ip_address' to the nearest 'lower_bound_ip_address'
# that is less than or equal to the user's IP.
df_merged = pd.merge_asof(
    df,
    ip_df,
    left_on='ip_address',
    right_on='lower_bound_ip_address',
    direction='backward'
)

# 4. Filter for validity
# merge_asof finds the *nearest* lower bound. We must ensure the IP is ALSO
# less than the upper bound of that country.
mask = (df_merged['ip_address'] <= df_merged['upper_bound_ip_address'])

# Create the final country column
df_merged['country'] = df_merged.loc[mask, 'country']

# Fill valid unmatched IPs with "Unknown"
df_merged['country'].fillna('Unknown', inplace=True)

# 5. Quick Check
print(f"\nmerged shape: {df_merged.shape}")
print(f"Countries found: {df_merged['country'].nunique()}")
print("\nTop 5 Countries by Transaction Count:")
print(df_merged['country'].value_counts().head())

# 6. Visualize Fraud by Country (Top 10) [cite: 382]
# We look at countries with at least 50 transactions to avoid noise
country_stats = df_merged.groupby('country').agg(
    total_transactions=('class', 'count'),
    fraud_rate=('class', 'mean')
).reset_index()

country_stats = country_stats[country_stats['total_transactions'] > 50]
top_fraud_countries = country_stats.sort_values(by='fraud_rate', ascending=False).head(10)

plt.figure(figsize=(12, 6))
sns.barplot(x='fraud_rate', y='country', data=top_fraud_countries, palette='Reds_r')
plt.title('Top 10 High-Risk Countries (by Fraud Rate)')
plt.xlabel('Fraud Rate')
plt.show()

In [ ]:
save_path = '../data/processed/fraud_data_merged.csv'

# Save to CSV
# index=False is crucial so we don't generate an extra column
df_merged.to_csv(save_path, index=False)

print(f"✅ Intermediate data saved to: {save_path}")
print(f"Shape: {df_merged.shape}")